In [1]:
import pandas as pd
import missingno 
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [2]:
df = pd.read_csv('main_df.csv')

In [3]:
df

,Unnamed: 0,User review,Review date,Star review,Text length,Has media,Has answer,Written by bot
0,0.0,Достоинства\r\n все устроило нормальная ...,2024-10-09,5,54,0,0,0
1,1.0,"Шикарная, начиталась отзывы, но рискнула, что ...",2024-10-09,5,195,0,0,0
2,2.0,Дочке понравилась,2024-10-09,5,17,0,0,0
3,3.0,"Пока не одевала ,но то что хотела",2024-10-09,5,33,0,0,0
4,4.0,Отличная куртка,2024-10-09,5,15,0,0,0
...,...,...,...,...,...,...,...,...
1901,203.0,"Все быстро, чётко и без проблем. Ставлю магази...",2024-11-03,5,55,1,0,True
1902,204.0,Отличный магазин! Теперь я ваш постоянный клие...,2024-11-02,5,56,0,1,True
1903,205.0,Широкий ассортимент и высокое качество обслужи...,2024-11-01,5,57,1,0,True
1904,206.0,"Всё чётко, оперативно и без нареканий. Доставк...",2024-10-31,5,57,1,1,True


In [122]:
df.drop(df.columns[0], axis=1, inplace=True)

In [123]:
df.head()

,User review,Review date,Star review,Text length,Has media,Has answer,Written by bot
0,Достоинства\r\n все устроило нормальная ...,2024-10-09,5,54,0,0,0
1,"Шикарная, начиталась отзывы, но рискнула, что ...",2024-10-09,5,195,0,0,0
2,Дочке понравилась,2024-10-09,5,17,0,0,0
3,"Пока не одевала ,но то что хотела",2024-10-09,5,33,0,0,0
4,Отличная куртка,2024-10-09,5,15,0,0,0


In [124]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1797 entries, 0 to 1796
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   User review     1797 non-null   object
 1   Review date     1797 non-null   object
 2   Star review     1797 non-null   int64 
 3   Text length     1797 non-null   int64 
 4   Has media       1797 non-null   int64 
 5   Has answer      1797 non-null   int64 
 6   Written by bot  1797 non-null   int64 
dtypes: int64(5), object(2)
memory usage: 98.4+ KB


In [125]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Star review,1797.0,4.495826,1.146608,1.0,5.0,5.0,5.0,5.0
Text length,1797.0,74.312187,75.599961,1.0,29.0,50.0,91.0,677.0
Has media,1797.0,0.237618,0.425743,0.0,0.0,0.0,0.0,1.0
Has answer,1797.0,0.641068,0.479821,0.0,0.0,1.0,1.0,1.0
Written by bot,1797.0,0.108514,0.311115,0.0,0.0,0.0,0.0,1.0


### Препроцессинг

In [126]:
df['Review date'] = pd.to_datetime(df['Review date'])
target = df["Written by bot"]
df_review = df['User review']
df_review_date = df['Review date']
df = df.drop(columns=['Written by bot', 'User review', 'Review date'])
df.head()

,Star review,Text length,Has media,Has answer
0,5,54,0,0
1,5,195,0,0
2,5,17,0,0
3,5,33,0,0
4,5,15,0,0


In [127]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1797 entries, 0 to 1796
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   Star review  1797 non-null   int64
 1   Text length  1797 non-null   int64
 2   Has media    1797 non-null   int64
 3   Has answer   1797 non-null   int64
dtypes: int64(4)
memory usage: 56.3 KB


### Избавление от выбросов

In [128]:
df_check = df['Text length']


Q1 = df_check.quantile(0.25)
Q3 = df_check.quantile(0.75)

IQR = Q3 - Q1 

lower_bond = Q1 - 1.5 * IQR 
upper_bond = Q3 + 1.5 * IQR 

df = df[(df['Text length'] >= lower_bond) & (df['Text length'] <= upper_bond)]
df.head()

,Star review,Text length,Has media,Has answer
0,5,54,0,0
2,5,17,0,0
3,5,33,0,0
4,5,15,0,0
5,5,24,0,0


In [129]:
df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 1661 entries, 0 to 1796
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   Star review  1661 non-null   int64
 1   Text length  1661 non-null   int64
 2   Has media    1661 non-null   int64
 3   Has answer   1661 non-null   int64
dtypes: int64(4)
memory usage: 64.9 KB


### Нормализация и стандартизация

In [130]:
minmax_scaler = MinMaxScaler()
standard_scaler = StandardScaler()

normalized_data = minmax_scaler.fit_transform(df)
df = pd.DataFrame(normalized_data, columns=df.columns)

standard_data = standard_scaler.fit_transform(df)
df= pd.DataFrame(standard_data, columns=df.columns)
df.head()

,Star review,Text length,Has media,Has answer
0,0.401612,-0.079442,-0.545579,-1.299116
1,0.401612,-0.994928,-0.545579,-1.299116
2,0.401612,-0.599042,-0.545579,-1.299116
3,0.401612,-1.044414,-0.545579,-1.299116
4,0.401612,-0.821728,-0.545579,-1.299116


In [131]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Star review,1661.0,1.026672e-16,1.000301,-3.410260,0.401612,0.401612,0.401612,0.401612
Text length,1661.0,-3.422241e-17,1.000301,-1.390814,-0.747500,-0.252642,0.563872,3.137130
Has media,1661.0,-2.994461e-17,1.000301,-0.545579,-0.545579,-0.545579,-0.545579,1.832916
Has answer,1661.0,-1.026672e-16,1.000301,-1.299116,-1.299116,0.769754,0.769754,0.769754


Да, можно последовательно использовать нормирование (например, с помощью MinMaxScaler) и затем стандартизацию (например, с помощью StandardScaler), но следует учитывать несколько моментов.

1. **Порядок применения**: Если вы сначала применяете нормирование, ваши данные будут иметь значения в диапазоне от 0 до 1. Если после этого применить стандартизацию, данные будут преобразованы так, что будут иметь среднее значение 0 и стандартное отклонение 1. Таким образом, стадия нормирования будет "перекрыта" этапом стандартизации.

2. **Необходимость**: Обычно нормирование и стандартизация служат разным целям в зависимости от требований задачи и модели. Например, некоторые алгоритмы (как SVM или KNN) чувствительны к масштабу данных, и для них может быть полезно использовать только одно из этих преобразований. Поэтому перед тем, как решать, нужно ли применять обе техники, подумайте о том, какая именно модель будет использоваться и какие свойства ваших данных вам важны.

3. **Универсальность**: Если у вас есть причины делать это в вашем конкретном случае (например, из-за специфических требований модели или данных), то ничего плохого в этом нет. Тем не менее, такое последовательное применение может вызвать ненужную сложность.

В целом, рекомендуется внимательно подходить к выбору методов предобработки данных и сосредотачиваться на понимании их влияния на конкретную задачу.